In [ ]:
### import libraries
import math
import pandas as pd
import numpy as np
import yfinance as yf
import requests as r
import concurrent.futures
import regex

In [ ]:
""" yfinance is a powerful tool that was created by a person a lot smarter than myself. 
Leveraging the code that was developed, and the Russells 2K Ticker list, 
I decided to scan the index for stocks with low PE's and High Divedend Yields.
You will have to gather the list on your own due to commercials that might need to be paid to get this list.
You can do this with other index's by 'scraping a page' for other indexes.
I will be doing this in the near future.
"""

# the below code will provide all of the available dimensions that can be pulled through yfinance.
# yf.Ticker('AAPL').info

In [ ]:
data = pd.read_csv('data/RTWO_Ticker_List.txt', sep=" ")
symbols = data
rty =[]
for symbol in symbols:
    stock = yf.Ticker(symbol)
    try:
        if stock.info['dividendYield']:
            rty.append(symbol + ',' + 
                    str(stock.info['currentPrice']) +
                    ',' + str(stock.info['dividendYield']) +
                    ',' + str(stock.info['averageVolume']) +
                    ',' + str(stock.info['sector']) +
                    ',' + str(stock.info['forwardPE']) +
                    ',' + str(stock.info['forwardEps']))
    except KeyError:
        pass

In [ ]:
rtypd = pd.Series(rty)
df = rtypd.str.split(',', expand=True)
df = df.rename(columns={0:'Symbol',1:'Last',2:'Div_Yld',3:'Avg_Vol',4:'Sector',5:'ForwardPE',6:'ForwardEPS'})
df=df.sort_values(by=['Div_Yld'], ascending=False)
df[['Last','Div_Yld','Avg_Vol']] = df[['Last','Div_Yld','Avg_Vol']].astype(float)
df2=df.drop(df[df['Div_Yld'] <= .075].index)
df3=df2.drop(df2[df2['Avg_Vol'] < 999999].index)
df4=df3.drop(df3[df3['Last'] < 5].index)
df4.to_excel('data/rty_5_2024.xlsx')

In [ ]:
# writing raw data to file
file1 = open('data/rty_5_24.csv', 'w')
file1.writelines(rty)
file1.close()

In [ ]:
# get yield function
def getyld(symbol):
    yldpct = None
    try:
        stock = yf.Ticker(symbol)
        yldpct = stock.info['dividendYield']
    except Exception:
        pass
    return(yldpct)